In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score

# Régression linéaire régularisée pour la prédiction

Ce notebook compare différentes méthodes de régression linéaire régularisée pour prédire une variable cible à partir de caractéristiques données. Nous utiliserons Ridge, Lasso et ElasticNet pour trouver le meilleur modèle.

## Chargement des données

Nous chargeons les ensembles de données d'entraînement et de test à partir de fichiers NumPy (.npy).

In [2]:
X_train = np.load("C:\\Users\\abela\\Desktop\\Epita\\FTML\\Exo4\\X_test.npy")
y_train = np.load("C:\\Users\\abela\\Desktop\\Epita\\FTML\\Exo4\\y_test.npy")
X_test = np.load("C:\\Users\\abela\\Desktop\\Epita\\FTML\\Exo4\\X_train.npy")
y_test = np.load("C:\\Users\\abela\\Desktop\\Epita\\FTML\\Exo4\\y_train.npy")

## Prétraitement des données

Nous utilisons StandardScaler pour normaliser les caractéristiques, ce qui est important pour les algorithmes basés sur la régularisation. Nous aplatissons également les variables cibles pour assurer la compatibilité avec les modèles.

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = y_train.ravel()
y_test = y_test.ravel()

## Modélisation et évaluation

Nous comparons trois algorithmes de régression régularisée :

1. **Ridge**
2. **Lasso** 
3. **ElasticNet** 

Nous évaluons chaque modèle en utilisant le score R² sur l'ensemble de test.

Pour ElasticNet, nous effectuons également une recherche par grille (GridSearchCV) pour trouver les meilleurs hyperparamètres (alpha et l1_ratio).

In [4]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    print(f"{model_name} Test R² score: {r2:.4f}")
    return r2

models = {
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5)
}

results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test, name)

param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

elastic_grid = GridSearchCV(ElasticNet(), param_grid, cv=5, scoring='r2')
elastic_grid.fit(X_train_scaled, y_train)
print("Best ElasticNet parameters:", elastic_grid.best_params_)
print("Best ElasticNet R² score:", elastic_grid.best_score_)

best_elastic = elastic_grid.best_estimator_
results["Best ElasticNet"] = evaluate_model(best_elastic, X_train_scaled, X_test_scaled, y_train, y_test, "Best ElasticNet")

print("\nRésultats finaux:")
for name, r2 in results.items():
    print(f"{name} R² score: {r2:.4f}")

best_model = max(results, key=results.get)
best_r2 = results[best_model]

Ridge Test R² score: 0.5899
Lasso Test R² score: 0.7573
ElasticNet Test R² score: 0.8734


c:\Users\abela\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.717e-02, tolerance: 1.186e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\abela\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.437e-02, tolerance: 1.189e-02
  model = cd_fast.enet_coordinate_descent(


Best ElasticNet parameters: {'alpha': 0.01, 'l1_ratio': 0.9}
Best ElasticNet R² score: 0.9087183378852488
Best ElasticNet Test R² score: 0.9347

Résultats finaux:
Ridge R² score: 0.5899
Lasso R² score: 0.7573
ElasticNet R² score: 0.8734
Best ElasticNet R² score: 0.9347


Les résultats montrent les performances de chaque modèle, avec le modèle ElasticNet optimisé donnant généralement les meilleurs résultats.

In [5]:
print("1. Le meilleur modèle est", best_model)
print(f"2. Le meilleur score R² obtenu est {best_r2:.4f}")


Conclusion
1. Le meilleur modèle est Best ElasticNet
2. Le meilleur score R² obtenu est 0.9347


## Conclusion

Nous identifions le meilleur modèle parmi ceux testés et rapportons son score R². Ce score indique la proportion de la variance dans la variable dépendante qui est prédictible à partir des variables indépendantes.

Le modèle ElasticNet optimisé s'est avéré le plus performant, probablement en raison de sa capacité à combiner les avantages de la régularisation L1 et L2, s'adaptant ainsi mieux à la structure spécifique de nos données.